In [2]:
import sys
import librosa
import librosa.display
import spotipy
import spotipy.util as util
from urllib.request import urlretrieve
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
%matplotlib inline

### Spotify Authorization

In [160]:
username='kaan3'
scope='user-library-read'
client_id='491a6bacf0324a5f9fae53586ed2d985'
client_secret='74f7c2bc50c54d86b8ba69e25ec638c5'
redirect_url='http://localhost/'
token=util.prompt_for_user_token(username,
                           scope,
                           client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirect_url)

### Define Functions

In [70]:
## get playlist and put it in a dataframe
def add_tracks(track_df,tracks):
    for i, item in enumerate(tracks['items']):
        track = item['track']
        if track['preview_url'] is None:
            preview=''
        else:    
            preview=track['preview_url']
        track_row=pd.Series([track['artists'][0]['name'],
                             track['name'],
                             track['duration_ms']/float(60000),
                             preview],
                            index=track_df.columns)
        track_df=track_df.append(track_row,ignore_index=True)
    return track_df

#get a single song 
def get_song(artist,song):
    df_columns=['Artist','Song','Duration','preview_url']
    sp=spotipy.Spotify(auth=token)
    results=sp.search(q='artist:'+artist+' track:'+song,limit=1)
    try:
        print('Preview URL found!')
        track_row=pd.Series([results['tracks']['items'][0]['artists'][0]['name'],
                             results['tracks']['items'][0]['name'],
                             results['tracks']['items'][0]['duration_ms']/float(60000),
                             results['tracks']['items'][0]['preview_url']],index=df_columns)
        return track_row
    except:
        print('There is no preview URL')
        return []
    

def get_playlist(username='kaan3',playlist_name='Puis danse'):
    #create empty track dataframe with column names
    df_columns=['Artist','Song','Duration','preview_url']
    track_df = pd.DataFrame(columns=df_columns)
    sp = spotipy.Spotify(auth=token)
    playlists=sp.user_playlists(username)
    selected_pl=[item for item in playlists['items'] if item.get('name')==playlist_name][0]
    total_tracks=selected_pl['tracks']['total']

    results = sp.user_playlist(username, selected_pl['id'],fields="tracks,next")
    tracks = results['tracks']
    track_df=add_tracks(track_df,tracks)
    print('%d / %d of tracks lack a preview URL' %(sum(x=='' for x in track_df['preview_url']),tracks['total']))
    return track_df

# Get rid of tracks w/o preview URLS's
def clean_nonePreview(track_df):
    clean_df=track_df[track_df['preview_url'] != '']
    clean_df.reset_index(inplace=True,drop=True)
    return clean_df


## Retrieve the previews and get desire features in a dataset
# tempo(from percussive)
# spectral bandwith difference (from harmonic) []
# average spectral flatness*100 [How much noise like a sound is]
# octave based spectral contrast** [represents the relative spectral distribution]
def analyze_track(row,analyze_columns=['tempo','band_diff','spec_flat']):
    url=row['preview_url']
    urlretrieve(url,'temp_song.wav')
    print('Preview of %s-%s successfully retrieved!' %(row['Artist'],row['Song']))
    print('Extracting Features...')
    
    y,sr = librosa.core.load('temp_song.wav')
    y_harmonic,y_percussive = librosa.effects.hpss(y)
    #tempo
    tempo,beat_frames = librosa.beat.beat_track(y_percussive,sr)
    #spectral bandwidth difference
    harmonic_band = librosa.feature.spectral_bandwidth(y=y_harmonic,sr=sr)
    band_diff = np.max(harmonic_band)-np.min(harmonic_band)
    #average specral flatness
    harmonic_flatness = librosa.feature.spectral_flatness(y=y_harmonic)
    spec_flat = np.mean(harmonic_flatness)*100
    #average spectral contrast
#     harmonic_contrast = librosa.feature.spectral_contrast(y=y_harmonic,sr=sr)
#     spec_contrast = np.mean(harmonic_contrast)
    print('Done')
    analyze_res=[tempo,band_diff,spec_flat]
#     for i,key in enumerate(analyze_dict):
#         analyze_dict[key]=analyze_res[i]
#     return analyze_dict
    analyze_series=pd.Series(analyze_res,index=analyze_columns)
    analyze_row=pd.concat([row,analyze_series],axis=0)
    return analyze_row

def analyze_playlist(playlist_df):
    playlist_df.reset_index(inplace=True,drop=True)
    analyze_columns=['tempo','band_diff','spec_flat']
    analyze_df=pd.DataFrame(columns=list(playlist_df.columns)+analyze_columns)
    for row in playlist_df.iterrows():
        try:
            row_result = analyze_track(row[1],analyze_columns)
            analyze_df = analyze_df.append(row_result,ignore_index=True)
        except:
            print('Preview of %s-%s does not exist, moving on ¯\_(ツ)_/¯' %(row[1]['Artist'],row[1]['Song']))
            pas
    return analyze_df

In [22]:
track=get_song('Bon Voyage Organisation','Love Soup')
track

Preview URL found!


Artist                                   Bon Voyage Organisation
Song                                                   Love Soup
Duration                                                  7.6191
preview_url    https://p.scdn.co/mp3-preview/da27df8b8d39c6d1...
dtype: object

In [92]:
final_track=analyze_track(track)
final_track

Preview of Bon Voyage Organisation-Love Soup successfully retrieved!
Extracting Features...
Done


Artist                                   Bon Voyage Organisation
Song                                                   Love Soup
Duration                                                  7.6191
preview_url    https://p.scdn.co/mp3-preview/da27df8b8d39c6d1...
tempo                                                    151.999
band_diff                                                1737.51
spec_flat                                               0.169476
dtype: object

In [80]:
track_df=get_playlist('kaan3','Puis danse')
clean_df1=clean_nonePreview(track_df)
clean_df1

5 / 32 of tracks lack a preview URL


Artist                            Song  Duration  \
0        Lewis OfMan                         Yo bene  3.629167   
1        Lewis OfMan                             Yes  5.048917   
2        Lewis OfMan                        Tes yeux  3.412917   
3           Elephanz                        Maryland  3.306883   
4           Elephanz            Blowing Like a Storm  3.319333   
5            Justice                       D.A.N.C.E  4.037767   
6        Lewis OfMan                           Flash  2.990283   
7           La Femme             Sur la planche 2013  3.830667   
8           La Femme                        Antitaxi  4.148883   
9      Clara Luciani                      La grenade  3.241550   
10          Las Aves                 Die in Shanghai  3.773767   
11  Vendredi sur Mer                   Écoute Chérie  3.772883   
12          Elephanz               Time for a Change  3.760500   
13           Paradis                      Toi Et Moi  3.680883   
14       Bleu Toucan                      Hanoï café  3.234700   
15        Polo & Pan  Plage isolée (Soleil couchant)  5.983917   
16   Kid Francescoli                    Les vitrines  3.294233   
17     L'Impératrice           Agitations tropicales  4.148200   
18          Elephanz           Bullitt - Bonus Track  3.319333   
19       Jabberwocky                       Honeymoon  4.023767   
20               Her                    Five Minutes  3.808433   
21            Corine   Pluie fine - Polo & Pan Remix  3.968017   
22    Weekend Affair                     Duel, pt. 2  3.929767   
23         Hypnolove                      La piscine  3.819100   
24         Hypnolove               Winter in the Sun  3.643767   
25        Polo & Pan                            Nanã  3.180433   
26          La Femme              Amour dans le motu  4.663767   

                                          preview_url  
0   https://p.scdn.co/mp3-preview/08408dd35288bdb6...  
1   https://p.scdn.co/mp3-preview/c49d367f3aea83a5...  
2   https://p.scdn.co/mp3-preview/d93183c56cee99ba...  
3   https://p.scdn.co/mp3-preview/073a63006711a9b5...  
4   https://p.scdn.co/mp3-preview/a8b8f6d09d70abc8...  
5   https://p.scdn.co/mp3-preview/49f7530531592515...  
6   https://p.scdn.co/mp3-preview/3dd92692c8c56451...  
7   https://p.scdn.co/mp3-preview/bb2fbf0a10906218...  
8   https://p.scdn.co/mp3-preview/6a1293d960170e81...  
9   https://p.scdn.co/mp3-preview/f4614444a448780e...  
10  https://p.scdn.co/mp3-preview/c5de6cbeea60576e...  
11  https://p.scdn.co/mp3-preview/12a398f6604154a4...  
12  https://p.scdn.co/mp3-preview/94fa5253894e84dd...  
13  https://p.scdn.co/mp3-preview/a135765e5a2b2de0...  
14  https://p.scdn.co/mp3-preview/204f15b6782d68db...  
15  https://p.scdn.co/mp3-preview/be209e8e9836980a...  
16  https://p.scdn.co/mp3-preview/eb5ec36e440859b4...  
17  https://p.scdn.co/mp3-preview/9f9a1220b4d95cc8...  
18  https://p.scdn.co/mp3-preview/cf04f34fb8b94a01...  
19  https://p.scdn.co/mp3-preview/31a2ff7df7f5c75e...  
20  https://p.scdn.co/mp3-preview/991a0a6a450c8669...  
21  https://p.scdn.co/mp3-preview/6ea146eaa4d80887...  
22  https://p.scdn.co/mp3-preview/af30d9b7a56d6c71...  
23  https://p.scdn.co/mp3-preview/12a79913097360c1...  
24  https://p.scdn.co/mp3-preview/c4de2416b0c716de...  
25  https://p.scdn.co/mp3-preview/f9b9a6d566760314...  
26  https://p.scdn.co/mp3-preview/7a019661bf6880c1...

In [83]:
final_df=analyze_playlist(clean_df1)

Preview of Lewis OfMan-Yo bene successfully retrieved!
Extracting Features...
Done
Preview of Lewis OfMan-Yes successfully retrieved!
Extracting Features...
Done
Preview of Lewis OfMan-Tes yeux successfully retrieved!
Extracting Features...
Done
Preview of Elephanz-Maryland successfully retrieved!
Extracting Features...
Done
Preview of Elephanz-Blowing Like a Storm successfully retrieved!
Extracting Features...
Done
Preview of Justice-D.A.N.C.E successfully retrieved!
Extracting Features...
Done
Preview of Lewis OfMan-Flash successfully retrieved!
Extracting Features...
Done
Preview of La Femme-Sur la planche 2013 successfully retrieved!
Extracting Features...
Done
Preview of La Femme-Antitaxi successfully retrieved!
Extracting Features...
Done
Preview of Clara Luciani-La grenade successfully retrieved!
Extracting Features...
Done
Preview of Las Aves-Die in Shanghai successfully retrieved!
Extracting Features...
Done
Preview of Vendredi sur Mer-Écoute Chérie successfully retrieved!
Ext

Artist                            Song  Duration  \
0        Lewis OfMan                         Yo bene  3.629167   
1        Lewis OfMan                             Yes  5.048917   
2        Lewis OfMan                        Tes yeux  3.412917   
3           Elephanz                        Maryland  3.306883   
4           Elephanz            Blowing Like a Storm  3.319333   
5            Justice                       D.A.N.C.E  4.037767   
6        Lewis OfMan                           Flash  2.990283   
7           La Femme             Sur la planche 2013  3.830667   
8           La Femme                        Antitaxi  4.148883   
9      Clara Luciani                      La grenade  3.241550   
10          Las Aves                 Die in Shanghai  3.773767   
11  Vendredi sur Mer                   Écoute Chérie  3.772883   
12          Elephanz               Time for a Change  3.760500   
13           Paradis                      Toi Et Moi  3.680883   
14       Bleu Toucan                      Hanoï café  3.234700   
15        Polo & Pan  Plage isolée (Soleil couchant)  5.983917   
16   Kid Francescoli                    Les vitrines  3.294233   
17     L'Impératrice           Agitations tropicales  4.148200   
18          Elephanz           Bullitt - Bonus Track  3.319333   
19       Jabberwocky                       Honeymoon  4.023767   
20               Her                    Five Minutes  3.808433   
21            Corine   Pluie fine - Polo & Pan Remix  3.968017   
22    Weekend Affair                     Duel, pt. 2  3.929767   
23         Hypnolove                      La piscine  3.819100   
24         Hypnolove               Winter in the Sun  3.643767   
25        Polo & Pan                            Nanã  3.180433   
26          La Femme              Amour dans le motu  4.663767   

                                          preview_url       tempo  \
0   https://p.scdn.co/mp3-preview/08408dd35288bdb6...  117.453835   
1   https://p.scdn.co/mp3-preview/c49d367f3aea83a5...  107.666016   
2   https://p.scdn.co/mp3-preview/d93183c56cee99ba...   92.285156   
3   https://p.scdn.co/mp3-preview/073a63006711a9b5...  103.359375   
4   https://p.scdn.co/mp3-preview/a8b8f6d09d70abc8...  123.046875   
5   https://p.scdn.co/mp3-preview/49f7530531592515...  112.347147   
6   https://p.scdn.co/mp3-preview/3dd92692c8c56451...  103.359375   
7   https://p.scdn.co/mp3-preview/bb2fbf0a10906218...   92.285156   
8   https://p.scdn.co/mp3-preview/6a1293d960170e81...   89.102909   
9   https://p.scdn.co/mp3-preview/f4614444a448780e...  129.199219   
10  https://p.scdn.co/mp3-preview/c5de6cbeea60576e...   95.703125   
11  https://p.scdn.co/mp3-preview/12a398f6604154a4...   92.285156   
12  https://p.scdn.co/mp3-preview/94fa5253894e84dd...  117.453835   
13  https://p.scdn.co/mp3-preview/a135765e5a2b2de0...  112.347147   
14  https://p.scdn.co/mp3-preview/204f15b6782d68db...   86.132812   
15  https://p.scdn.co/mp3-preview/be209e8e9836980a...   95.703125   
16  https://p.scdn.co/mp3-preview/eb5ec36e440859b4...  112.347147   
17  https://p.scdn.co/mp3-preview/9f9a1220b4d95cc8...  117.453835   
18  https://p.scdn.co/mp3-preview/cf04f34fb8b94a01...  161.499023   
19  https://p.scdn.co/mp3-preview/31a2ff7df7f5c75e...  117.453835   
20  https://p.scdn.co/mp3-preview/991a0a6a450c8669...   99.384014   
21  https://p.scdn.co/mp3-preview/6ea146eaa4d80887...  117.453835   
22  https://p.scdn.co/mp3-preview/af30d9b7a56d6c71...  117.453835   
23  https://p.scdn.co/mp3-preview/12a79913097360c1...  123.046875   
24  https://p.scdn.co/mp3-preview/c4de2416b0c716de...  123.046875   
25  https://p.scdn.co/mp3-preview/f9b9a6d566760314...  123.046875   
26  https://p.scdn.co/mp3-preview/7a019661bf6880c1...  151.999081   

      band_diff  spec_flat  
0   1939.289537   0.223707  
1   3189.320402   0.257477  
2   1792.949728   0.059923  
3   1681.799183   1.653450  
4   2905.989992   0.973089  
5   2026.356554   0.973387  
6   1868.355383   0.471937  


In [86]:
f1=(final_df,'Puis danse')

In [93]:
df_list=[f1]
track_list=[final_track]

In [125]:
for track in track_list:
    print([track['tempo']])
    print([track['band_diff']])
    print([track['spec_flat']])

[151.99908088235293]
[1737.5125689653316]
[0.16947552794590592]


In [152]:
#Plotting
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='kaancet', api_key='U399IWwsVmePOzBGwRST')
plotly.offline.init_notebook_mode()
from plotly.offline import iplot
from ipywidgets import widgets
from IPython.display import display, clear_output, Image
from ipywidgets import interact, interactive, fixed, interact_manual
from plotly.widgets import GraphWidget

### Define widgets

In [188]:
username_text = widgets.Text(
    value = '',
    description = 'Username:',
    disabled=False,
    continuous_update=False
    )

playlist_drop = widgets.Dropdown(
    description = 'Playlist:',
    disabled=False,
    continuous_update=False
    )


get_playlist_button = widgets.Button(
    value=False,
    description='Get Playlist',
    disabled=False
    )

display(username_text)
display(playlist_drop)
display(get_playlist_button)

def text_search(change):
    sp = spotipy.Spotify(auth=token)
    playlists=sp.user_playlists(username_text.value)
    playlist_names=[]
    for item in playlists['items']:
        print(item.get('name'))
        playlist_names.append(item.get('name'))
    playlist_drop.options=playlist_names
    
def playlist_select(change):
    print(playlist_drop.value)

def on_button_clicked(b):
    name=username_text.value
    pl=playlist_drop.value
    df=get_playlist(name,pl)
    out_df=clean_nonePreview(df)
    print(out_df)
    
username_text.observe(text_search,names='value')
playlist_drop.observe(playlist_select,names='value')
get_playlist_button.on_click(on_button_clicked)

Text(value='', continuous_update=False, description='Username:')

Dropdown(description='Playlist:', options=(), value=None)

Button(description='Get Playlist', style=ButtonStyle())

❄️💊
ｌｉｓｔｅｎ　ｈｅｒｅ　ｕ　ｌｉｌ　ｓｈｉｔ
Puis danse
RIP Vinyl aka NO SKIP
EZ  👂🏼
Kukumba
gnj
❄️💊
Puis danse
5 / 32 of tracks lack a preview URL
              Artist                            Song  Duration  \
0        Lewis OfMan                         Yo bene  3.629167   
1        Lewis OfMan                             Yes  5.048917   
2        Lewis OfMan                        Tes yeux  3.412917   
3           Elephanz                        Maryland  3.306883   
4           Elephanz            Blowing Like a Storm  3.319333   
5            Justice                       D.A.N.C.E  4.037767   
6        Lewis OfMan                           Flash  2.990283   
7           La Femme             Sur la planche 2013  3.830667   
8           La Femme                        Antitaxi  4.148883   
9      Clara Luciani                      La grenade  3.241550   
10          Las Aves                 Die in Shanghai  3.773767   
11  Vendredi sur Mer                   Écoute Chérie  3.772883   
12          

In [142]:
def plotAll(df_list=[],track_list=[]):
    data=[]
    hex_color=['0','1','2','3','4','5','6','7','A','B','C','D','E','F']
    color_list = ["#"+''.join([np.random.choice(hex_color) for j in range(6)])for i in range(len(df_list))]
    for i,playlist in enumerate(df_list):
        trace=go.Scatter3d(
            x=playlist[0]['tempo'],
            y=playlist[0]['band_diff'],
            z=playlist[0]['spec_flat'],
            text=playlist[0]['Song'],
            mode='markers',
            marker=dict(
                size=8,
                color=color_list[i]
                ),
            opacity=0.9,
            name=playlist[1]
        )
        data.append(trace)
        
    for track in track_list:
        track_trace=go.Scatter3d(
            x=[track['tempo']],
            y=[track['band_diff']],
            z=[track['spec_flat']],
            text=track['Song'],
            mode='markers',
            marker=dict(
                size=8,
                color='#af2427',
                line=dict(
                    color='#27af24',
                    width=0.5
                ),
            opacity=0.9,
            ),
            name=track['Song']
        )
        data.append(track_trace)
    
    # layout of the figure
    layout=go.Layout(
        scene = dict(
        xaxis=dict(
            title='Tempo',
            titlefont=dict(
                family='Helvetica',
                size=18,
                color='#585958'
            ),
#             range=[]
        ),
        yaxis=dict(
            title='Bandwidth Difference',
            titlefont=dict(
                family='Helvetica',
                size=18,
                color='#585958'
            ),
#             range=[]
        ),
        zaxis=dict(
            title='Spectral Flatness',
            titlefont=dict(
                family='Helvetica',
                size=18,
                color='#585958'
            ),
#             range=[]
        )),
        autosize=False,
        width=800,
        height=800,
    )

    fig=go.Figure(data=data,layout=layout)
    return py.iplot(fig,filename='deneme22')  

In [143]:
plotAll(df_list,track_list)